# **What genres do the best in theatres?**
- Motivation:
    - Creating movies in popular genres will attract more initial viewers. It’s what the people are interested in watching.


**Smaller questions!**
- Which Ratings generate the most money?
- Which genre is the least lucrative ?
    - Which movie had the best investment return ratio? And which Genres normally do well ?
    - Which movies generated money only in their country of origin?

In [39]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

Assign all the CSV files to something and add comments to it

In [40]:
#Titles, Studio, Domestic Gross, Foreign Gross and Year
#3387 rows
movies_gross = pd.read_csv("zippedData/bom.movie_gross.csv")

#Nconst, Primary Name, Primary Profession, Known for titles(title_id)
#606648 rows
basics_names = pd.read_csv("zippedData/name.basics.csv")

#Title ID's, Ordering, Title, Region, Language, Types, Attributes, Original Title
#331703 rows
akas = pd.read_csv("zippedData/title.akas.csv")

#Tconst, Primary Title, Original Title, Start Year, Runtime Minutes, Genres
#146144 rows
basics_titles = pd.read_csv("zippedData/title.basics.csv")

#Tconst, Directors, Writers
#146144 rows
crew = pd.read_csv("zippedData/title.crew.csv")

#Tconst, Ordering, Nconst, Category, Job, Characters
#1028186 rows
principals = pd.read_csv("zippedData/title.principals.csv")

#Tconst, Average Rating, Number of Votes
#73856 rows
ratings = pd.read_csv("zippedData/title.ratings.csv")

#Genre Id, Id, Original Language, Original Language, Original Title, Popularity, Release Date, Title, Vote Average, Vote count 
#26517 rows
movies = pd.read_csv("zippedData/tmdb.movies.csv")

#Id, Release Date, Movie Name, Production Budget, Domestic Gross, Worldwide Gross
#5782 rows
movies_budgets = pd.read_csv("zippedData/tn.movie_budgets.csv")

# tconst = Movie
# nconst = Person

Assign all the TSV files to something and add comments to it

In [41]:
#Id, Synopsis, Rating, Genre, Director, Writer, Theater Date, Dvd Date, Currency, Box Office, Runtime, Studio
#1560 rows 
movie_info = pd.read_csv("zippedData/rt.movie_info.tsv", sep='\t')

#Id, Reviews, Rating, Fresh, Critic, Top Critic, Publisher, Date
#54432 rows
reviews = pd.read_csv("zippedData/rt.reviews.tsv", sep='\t', encoding = 'unicode_escape')


2009 from 2019, treat 2020 as a special year


In [73]:
# Creating a table with only the rating, how much it made and it's genre
# Cleaning NAN values
# rating_genre_box = movie_info.drop(['id','currency','director', 'synopsis', 'writer', 'theater_date', 'dvd_date', 'runtime', 'studio'], axis=1)
# rating_genre_box.dropna(inplace=True)
### Impossible to use, ID is useless!

# Working with the basics_titles dataframe

Creating a table that contains Titles, Genres, Start Year and Tconst (cleaning some columns)

In [42]:
# Cleaning basics_titles to only those released after 2008
# Cleaning again to only those released before 2020
# Cleaning NAN values
# Using split to create a new column with separeted Genres
# Droping the unused Genres columns

a = basics_titles[basics_titles['start_year'] >= 2009]
b = a[a['start_year'] <= 2019]
title_genre = b.drop(['original_title', 'runtime_minutes'], axis=1)
title_genre.dropna(inplace=True)
# title_genre['genre_new'] = title_genre['genres'].str.split(",")
# title_genre.drop('genres', axis=1, inplace=True)

In [43]:
title_genre.head()

,tconst,primary_title,start_year,genres
0,tt0063540,Sunghursh,2013,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,2019,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,2018,Drama
3,tt0069204,Sabse Bada Sukh,2018,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,2017,"Comedy,Drama,Fantasy"


Creating a new table with the rating, tconst, primary_title and genre

In [69]:
#73856 rows

titlegenrerating = ratings.join(title_genre.set_index('tconst'), on='tconst')
titlegenrerating.head(3)

,tconst,averagerating,numvotes,primary_title,start_year,genres
0,tt10356526,8.3,31,Laiye Je Yaarian,2019.0,Romance
1,tt10384606,8.9,559,Borderless,2019.0,Documentary
2,tt1042974,6.4,20,Just Inès,2010.0,Drama


Trying to integrate "title_genre" with "ratings" to measure sucess based on ratings



Still trying to figure out a way to make the genre works 
    - How to divide into 1 when have multiples? 
    - How to join the tables?

How to measure sucess? Based on how much money/investment? Money? Or reviews? Or a mix of the two?

# Working with the movies_budgets dataframe

The goal here is to create a table with Movies Name, Budget, Domestic Gross and Worldwide Gross(Cleaning the column ID)

In [45]:
# Cleaning NAN values
# Renaming Column to match the other table, to make left-join possible

movies_money = movies_budgets.drop(['id', 'release_date'], axis=1)
movies_money.dropna(inplace=True)
movies_money.rename(columns={"movie": "primary_title"}, inplace=True)

In [46]:
#5782 rows
movies_money.head(2)

,primary_title,production_budget,domestic_gross,worldwide_gross
0,Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"


In [68]:
# 139.722 rows
title_genre.head(2)

,tconst,primary_title,start_year,genres
0,tt0063540,Sunghursh,2013,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,2019,"Biography,Drama"


In [48]:
# Using method str.replace to remove ',' and '$' from the data columns 
# Using astype to change the values in those columns from String to Float because to Int gives an error

movies_money['worldwide_gross'] = movies_money['worldwide_gross'].str.replace(',', '')
movies_money['worldwide_gross'] = movies_money['worldwide_gross'].str.replace('$', '')
movies_money['worldwide_gross'] = movies_money['worldwide_gross'].astype(float)

movies_money['production_budget'] = movies_money['production_budget'].str.replace(',', '')
movies_money['production_budget'] = movies_money['production_budget'].str.replace('$', '')
movies_money['production_budget'] = movies_money['production_budget'].astype(float)

movies_money['domestic_gross'] = movies_money['domestic_gross'].str.replace(',', '')
movies_money['domestic_gross'] = movies_money['domestic_gross'].str.replace('$', '')
movies_money['domestic_gross'] = movies_money['domestic_gross'].astype(float)


In [50]:
# Creating a new column to give the investment return (How many dollars you get per dollar invested)
# Checking for duplicated entries on the data based on the primary title column q
# Dropping all the duplicates, not keeping any so it doesn't conflict with the other table and we get false information
# Checking one last time to see if we still have duplicates

movies_money['investment_return'] = movies_money['worldwide_gross']/movies_money['production_budget']
movies_money['investment_return'] = movies_money['investment_return'] - 1

duplicates_money = movies_money[movies_money.duplicated(subset=['primary_title'],keep=False)].sort_values(['primary_title'])
duplicates_money.head(6)

movies_money.drop_duplicates(subset='primary_title', keep=False, inplace=True)


In [67]:
movies_money.head(2)

,primary_title,production_budget,domestic_gross,worldwide_gross,investment_return
0,Avatar,425000000.0,760507625.0,2.776345e+09,5.532577
1,Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,1.546673


# Creating the new table moviesfullinfo

The idea here is to join two tables to make one with ratings and money to make mostly a correlation between these 2 metrics

In [55]:
# Dropping Tconst, since this one is supposed to be the final table to use for graphs
# Dropping nan's values
# Creating a filter to only movies that had a budget over a million dollars

moviesfullinfo = movies_money.join(titlegenrerating.set_index('primary_title'), on='primary_title')
moviesfullinfo.drop(['tconst'], axis=1, inplace=True)
moviesfullinfo = moviesfullinfo.dropna()
moviesfullinfo = moviesfullinfo[moviesfullinfo['production_budget'] >= 1000000]

# added
movieswoutratings = movieswoutratings[movieswoutratings['production_budget'] >= 1000000]
movieswoutratings = movieswoutratings[movieswoutratings['worldwide_gross'] >= 1]

In [59]:
#2410 rows
moviesfullinfo.head()

,primary_title,production_budget,domestic_gross,worldwide_gross,investment_return,averagerating,numvotes,start_year,genres
0,Avatar,425000000.0,760507625.0,2.776345e+09,5.532577,6.1,43.0,2011.0,Horror
1,Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,1.546673,6.6,447624.0,2011.0,"Action,Adventure,Fantasy"
2,Dark Phoenix,350000000.0,42762350.0,1.497624e+08,-0.572108,6.0,24451.0,2019.0,"Action,Adventure,Sci-Fi"
3,Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09,3.243841,7.3,665594.0,2015.0,"Action,Adventure,Sci-Fi"
6,Avengers: Infinity War,300000000.0,678815482.0,2.048134e+09,5.827114,8.5,670926.0,2018.0,"Action,Adventure,Sci-Fi"


# Creating the new table movieswoutratings

Here the goal was to make a table to make a correlation between Genre, Production, Worldwide Gross, Domestic Gross and ROI's

In [64]:
# Using method str.replace to remove ',' and '$' from the data columns 
# Using astype to change the values in those columns from String to Float because to Int gives an error

movieswoutratings = moviesfullinfo
movieswoutratings = movieswoutratings.dropna()

movieswoutratings = movieswoutratings[movieswoutratings['production_budget'] >= 1000000]
movieswoutratings = movieswoutratings[movieswoutratings['worldwide_gross'] >= 1]



In [65]:
#3392 rows
movieswoutratings.head(3)

,primary_title,production_budget,domestic_gross,worldwide_gross,investment_return,averagerating,numvotes,start_year,genres
0,Avatar,425000000.0,760507625.0,2.776345e+09,5.532577,6.1,43.0,2011.0,Horror
1,Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,1.546673,6.6,447624.0,2011.0,"Action,Adventure,Fantasy"
2,Dark Phoenix,350000000.0,42762350.0,1.497624e+08,-0.572108,6.0,24451.0,2019.0,"Action,Adventure,Sci-Fi"


In [19]:
# genres_test = movieswoutratings['genre_new']
action = moviesfullinfo[moviesfullinfo['genres'].str.contains("Action")]
adventure = moviesfullinfo[moviesfullinfo['genres'].str.contains("Adventure")]
animation = moviesfullinfo[moviesfullinfo['genres'].str.contains("Animation")]
biography = moviesfullinfo[moviesfullinfo['genres'].str.contains("Biography")]
crime = moviesfullinfo[moviesfullinfo['genres'].str.contains("Crime")]
documentary = moviesfullinfo[moviesfullinfo['genres'].str.contains("Documentary")]
drama = moviesfullinfo[moviesfullinfo['genres'].str.contains("Drama")]
family = moviesfullinfo[moviesfullinfo['genres'].str.contains("Family")]
fantasy = moviesfullinfo[moviesfullinfo['genres'].str.contains("Fantasy")]
history = moviesfullinfo[moviesfullinfo['genres'].str.contains("History")]
horror = moviesfullinfo[moviesfullinfo['genres'].str.contains("Horror")]
music = moviesfullinfo[moviesfullinfo['genres'].str.contains("Music")]
musical = moviesfullinfo[moviesfullinfo['genres'].str.contains("Musical")]
mystery = moviesfullinfo[moviesfullinfo['genres'].str.contains("Mystery")]
romance = moviesfullinfo[moviesfullinfo['genres'].str.contains("Romance")]
scifi = moviesfullinfo[moviesfullinfo['genres'].str.contains("Sci-Fi")]
sport = moviesfullinfo[moviesfullinfo['genres'].str.contains("Sport")]
thriller = moviesfullinfo[moviesfullinfo['genres'].str.contains("Thriller")]
war = moviesfullinfo[moviesfullinfo['genres'].str.contains("War")]
western = moviesfullinfo[moviesfullinfo['genres'].str.contains("Western")]

# Action
# Adventure
# Animation
# Biography
# Crime
# Documentary
# Drama
# Family
# Fantasy
# History
# Horror
# Music - so for Musical just see if it contains Music
# Musical
# Mystery
# Romance
# Sci-Fi
# Sport
# Thriller
# War
# Western


In [20]:
western

,primary_title,production_budget,domestic_gross,worldwide_gross,investment_return,averagerating,numvotes,start_year,genres
12,The Lone Ranger,275000000.0,89302115.0,260002115.0,-0.054538,6.4,208827.0,2013.0,"Action,Adventure,Western"
368,Django Unchained,100000000.0,162805434.0,449948323.0,3.499483,8.4,1211405.0,2012.0,"Drama,Western"
454,The Magnificent Seven,90000000.0,93432655.0,162525156.0,0.805835,6.9,168514.0,2016.0,"Action,Adventure,Western"
793,The Kid,65000000.0,69688384.0,69688384.0,0.072129,6.0,1573.0,2019.0,"Biography,Drama,Western"
951,The Ridiculous 6,60000000.0,0.0,0.0,-1.000000,4.8,38694.0,2015.0,"Comedy,Western"
1364,The Warrior's Way,42000000.0,5666340.0,8514325.0,-0.797278,6.3,24271.0,2010.0,"Action,Fantasy,Western"
1628,True Grit,35000000.0,171243005.0,252276928.0,6.207912,7.6,284034.0,2010.0,"Adventure,Drama,Western"
2311,Jane Got a Gun,25000000.0,1513793.0,3771345.0,-0.849146,5.9,19051.0,2015.0,"Action,Drama,Western"
2853,Forsaken,18000000.0,0.0,0.0,-1.000000,6.3,9199.0,2015.0,"Action,Drama,Western"
2969,The Homesman,16000000.0,2429989.0,8217571.0,-0.486402,6.6,27724.0,2014.0,"Drama,Western"


# To Do List TODAY

Use ratings to assimilate which **genre** are the most likely to do best with the critics

- Usar ratings pra assimilar quais sao os generos que normalmente desempenham melhor com a critica

Try to go with scatter plot with legends OR Bar Chart with ROI's?